<a href="https://colab.research.google.com/github/Huy1902/HAC/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
from tqdm import tqdm
import os
from IPython.display import display

In [6]:
with open('rl4rs/rl4rs_dataset_b_sl.csv', 'r') as f:
    data = f.read().split('\n')[1:-1]
    print('number of records: ', len(data))
    print('number of users: ', len(set([x.split('@')[1] for x in data])))
    print('number of items per page: ', len(data[0].split('@')[3].split(',')))
    print('avg. lengths of user history: ', sum([len(x.split('@')[5].split(',')) for x in data])/len(data))
    print('avg. dimensions of user_protrait: ', sum([len(x.split('@')[6].split(',')) for x in data])/len(data))
    print('avg. dimensions of item_feature: ', sum([len(x.split('@')[7].split(',')) for x in data])/len(data))
    print('number of behavior_policy_id: ', len(set([x.split('@')[-1] for x in data])))

number of records:  937949
number of users:  519435
number of items per page:  9
avg. lengths of user history:  36.35307676643399
avg. dimensions of user_protrait:  42.0
avg. dimensions of item_feature:  352.0
number of behavior_policy_id:  1


In [8]:
with open('rl4rs/rl4rs_dataset_a_sl.csv', 'r') as f:
    data = f.read().split('\n')[1:-1]
    print('number of records: ', len(data))
    print('number of users: ', len(set([x.split('@')[1] for x in data])))
    print('number of items per page: ', len(data[0].split('@')[3].split(',')))
    print('avg. lengths of user history: ', sum([len(x.split('@')[5].split(',')) for x in data])/len(data))
    print('avg. dimensions of user_protrait: ', sum([len(x.split('@')[6].split(',')) for x in data])/len(data))
    print('avg. dimensions of item_feature: ', sum([len(x.split('@')[7].split(',')) for x in data])/len(data))
    print('number of behavior_policy_id: ', len(set([x.split('@')[-1] for x in data])))

number of records:  937949
number of users:  937949
number of items per page:  9
avg. lengths of user history:  36.35307676643399
avg. dimensions of user_protrait:  42.0
avg. dimensions of item_feature:  352.0
number of behavior_policy_id:  1


In [12]:
with open('rl4rs/rl4rs_dataset_a_rl.csv', 'r') as f:
    data = f.read().split('\n')[1:-1]
    print('number of records: ', len(data))
    print('number of users: ', len(set([x.split('@')[1] for x in data])))
    print('number of items per page: ', len(data[0].split('@')[3].split(',')))
    print('avg. lengths of user history: ', sum([len(x.split('@')[5].split(',')) for x in data])/len(data))
    print('avg. dimensions of user_protrait: ', sum([len(x.split('@')[6].split(',')) for x in data])/len(data))
    print('avg. dimensions of item_feature: ', sum([len(x.split('@')[7].split(',')) for x in data])/len(data))
    print('number of behavior_policy_id: ', len(set([x.split('@')[-1] for x in data])))
    

number of records:  781367
number of users:  781367
number of items per page:  9
avg. lengths of user history:  48.076561974078764
avg. dimensions of user_protrait:  42.0
avg. dimensions of item_feature:  352.0
number of behavior_policy_id:  1


In [35]:
with open('rl4rs/rl4rs_dataset_b_rl.csv', 'r') as f:
    data = f.read().split('\n')[1:-1]
    print('number of records: ', len(data))
    print('number of users: ', len(set([x.split('@')[1] for x in data])))
    print('number of items per page: ', len(data[0].split('@')[3].split(',')))
    print('avg. lengths of user history: ', sum([len(x.split('@')[5].split(',')) for x in data])/len(data))
    print('avg. dimensions of user_protrait: ', sum([len(x.split('@')[6].split(',')) for x in data])/len(data))
    print('avg. dimensions of item_feature: ', sum([len(x.split('@')[7].split(',')) for x in data])/len(data))
    print('number of behavior_policy_id: ', len(set([x.split('@')[-1] for x in data])))

number of records:  781367
number of users:  439131
number of items per page:  9
avg. lengths of user history:  48.076561974078764
avg. dimensions of user_protrait:  42.0
avg. dimensions of item_feature:  352.0
number of behavior_policy_id:  1


In [26]:
session = [entry.split('@')[1] for entry in data]
print(session[:5])
session = list(set(session))
print(f"Number of unique sessions: {len(session)}")
# Each user has a unique session id, so we can use this to identify users.

['1', '1', '2', '3', '4']
Number of unique sessions: 439131


In [38]:
path_to_data = 'rl4rs/rl4rs_dataset_b_rl.csv'
path_to_item = 'rl4rs/item_info.csv'
path_to_output = "Hyper-Actor-Critic-for-Recommendation/dataset/rl4rs/"
seed = 2025

## Extract train and test dataset

In [37]:
# Split the data into training (80%) and evaluation (20%) sets based on timestamp
split_index = int(len(data) * 0.8)

# Training data
train_data = data[:split_index]

# Evaluation data
eval_data = data[split_index:]

print(f"Training data size: {len(train_data)}")
print(f"Evaluation data size: {len(eval_data)}")

Training data size: 625093
Evaluation data size: 156274


In [44]:
import csv
import os

header = ['timestamp', 'session_id', 'sequence_id', 'exposed_items', 'user_feedback', 'user_seqfeature', 'user_protrait', 'item_feature', 'behavior_policy_id']

# Split each record into columns and save to CSV
train_path = os.path.join(path_to_output, 'train.csv')
test_path = os.path.join(path_to_output, 'test.csv')

with open(train_path, 'w', newline='', encoding='utf-8') as train_file:
  writer = csv.writer(train_file, delimiter='@')
  writer.writerow(header)
  for record in train_data:
    writer.writerow(record.split('@'))

with open(test_path, 'w', newline='', encoding='utf-8') as test_file:
  writer = csv.writer(test_file, delimiter='@')
  writer.writerow(header)
  for record in eval_data:
    writer.writerow(record.split('@'))